In [91]:
import pandas as pd
import numpy as np

In [92]:
#read in csv
df= pd.read_csv("csvs/AnalysisDataset.csv")

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37851 entries, 0 to 37850
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        37851 non-null  int64  
 1   category          35219 non-null  object 
 2   item              37847 non-null  object 
 3   rating            37851 non-null  float64
 4   cleanedText       37700 non-null  object 
 5   timestamp         37851 non-null  object 
 6   numRating         37602 non-null  float64
 7   average_rating    37851 non-null  float64
 8   total_votes       37851 non-null  int64  
 9   sentiment         37851 non-null  float64
 10  popularity_score  37602 non-null  float64
dtypes: float64(5), int64(2), object(4)
memory usage: 3.2+ MB


In [94]:
df.head(5)

,Unnamed: 0,category,item,rating,cleanedText,timestamp,numRating,average_rating,total_votes,sentiment,popularity_score
0,1,All Beauty,Organic Bamboo Cotton Ear Swabs by Bali Boo - ...,5.0,really like ear swab first come large handy bo...,2020-08-03 16:49:11.088,49.0,4.2,0,1.0,30.66
1,2,All Beauty,Iryasa Night Indulge Cream - Natural Face Crea...,5.0,honest rarely used overnight cream typically s...,2020-06-28 13:53:42.858,38.0,3.7,1,1.0,24.01
2,4,All Beauty,1/8 inch Snap On Rollers 8 Pack (Pink),5.0,purchased see could set mom hair fine thin sho...,2020-08-08 15:44:00.832,23.0,4.3,0,1.0,15.09
3,5,All Beauty,Cotton Headbands 6 Pack Stretch Elastic Yoga S...,5.0,great multi pack band great price wash well la...,2017-02-01 13:32:35.000,2253.0,4.3,0,1.0,1353.09
4,6,All Beauty,KeyZone Waterproof Drawing Eye Brow Eyebrow Pe...,5.0,easy use,2019-06-28 16:10:44.885,52.0,3.5,0,1.0,32.25


In [95]:
#changing timestamp to type datetime
df['timestamp']= pd.to_datetime(df['timestamp'])

In [96]:
#take the month and create new column
df['month']= df['timestamp'].dt.month

In [97]:
#group the categories by month and get a count of each review per category per month and positive sentiment count (just sum cause negative is 0 anyway)
group= df.groupby(['category', 'month']).agg(totalReviews=('sentiment', 'count'), positive=('sentiment', 'sum')).reset_index()

In [98]:
#calculate positive sentiment ratio
group['ratio']= group['positive'] / group['totalReviews']
group['ratio']= group['ratio'].round(2)
group

,category,month,totalReviews,positive,ratio
0,AMAZON FASHION,1,88,79.0,0.90
1,AMAZON FASHION,2,57,51.0,0.89
2,AMAZON FASHION,3,83,77.0,0.93
3,AMAZON FASHION,4,79,75.0,0.95
4,AMAZON FASHION,5,70,65.0,0.93
...,...,...,...,...,...
396,Video Games,8,75,69.0,0.92
397,Video Games,9,61,51.0,0.84
398,Video Games,10,66,60.0,0.91
399,Video Games,11,78,66.0,0.85


In [99]:
#create a new dataframe pivoting the columns to rows and rows to columns
groupMonth= group.pivot(index= 'category', columns= 'month', values= 'totalReviews').fillna(0)
groupRatio= group.pivot(index= 'category', columns= 'month', values= 'ratio').fillna(0)

In [100]:
groupMonth.head(1)

month,1,2,3,4,5,6,7,8,9,10,11,12
category,,,,,,,,,,,,
AMAZON FASHION,88.0,57.0,83.0,79.0,70.0,63.0,64.0,60.0,59.0,75.0,54.0,69.0


In [101]:
groupRatio.head(1)

month,1,2,3,4,5,6,7,8,9,10,11,12
category,,,,,,,,,,,,
AMAZON FASHION,0.9,0.89,0.93,0.95,0.93,0.92,0.78,0.93,0.9,0.91,0.87,0.91


In [102]:
months= range(1, 13) #1 -> 12
month_names= ['January', 'February', 'March', 'April', 'May', 'June', 
              'July', 'August', 'September', 'October', 'November', 'December']

#reindexs to make category not the index anymore
groupMonth= groupMonth.reindex(columns= months, fill_value= 0.0).astype(float)
groupRatio= groupRatio.reindex(columns= months, fill_value= 0).astype(float)


In [103]:
groupMonth.head(1)

month,1,2,3,4,5,6,7,8,9,10,11,12
category,,,,,,,,,,,,
AMAZON FASHION,88.0,57.0,83.0,79.0,70.0,63.0,64.0,60.0,59.0,75.0,54.0,69.0


In [104]:
groupRatio.head(1)

month,1,2,3,4,5,6,7,8,9,10,11,12
category,,,,,,,,,,,,
AMAZON FASHION,0.9,0.89,0.93,0.95,0.93,0.92,0.78,0.93,0.9,0.91,0.87,0.91


In [105]:
# concat and rename columns
totalRatio= pd.concat([
    groupMonth.rename(columns= dict(zip(months, [f"{i}_Total"for i in month_names]))),
    groupRatio.rename(columns= dict(zip(months, [f"{i}_Ratio"for i in month_names])))
], axis= 1)


In [106]:
columnOrder= []
for i in month_names:
    columnOrder.extend([f"{i}_Total", f"{i}_Ratio"])

In [107]:
totalRatio= totalRatio[columnOrder].reset_index()

In [108]:
#make total columns integers so no decimal
totalCols= [col for col in totalRatio.columns if '_Total' in col]
totalRatio[totalCols]= totalRatio[totalCols].astype(int)

Now the table has the total reviews for the month(popularity) and the sentiment ratio for the month (0.9 means that 90% of the reviews were positive). 
<br/>
<br/>
This means that we can see if an item was popular because it was good or popular because it was bad

In [109]:
totalRatio

month,category,January_Total,January_Ratio,February_Total,February_Ratio,March_Total,March_Ratio,April_Total,April_Ratio,May_Total,...,August_Total,August_Ratio,September_Total,September_Ratio,October_Total,October_Ratio,November_Total,November_Ratio,December_Total,December_Ratio
0,AMAZON FASHION,88,0.90,57,0.89,83,0.93,79,0.95,70,...,60,0.93,59,0.90,75,0.91,54,0.87,69,0.91
1,All Beauty,115,0.88,95,0.87,115,0.91,100,0.81,110,...,116,0.84,82,0.82,83,0.87,88,0.90,107,0.83
2,All Electronics,42,0.95,29,0.83,17,0.47,16,0.88,15,...,11,0.82,22,0.91,16,1.00,17,0.94,32,0.94
3,Amazon Devices,4,0.50,1,1.00,0,0.00,3,1.00,1,...,3,1.00,1,1.00,0,0.00,2,1.00,3,1.00
4,Amazon Home,191,0.93,137,0.91,146,0.91,148,0.94,142,...,135,0.90,108,0.93,153,0.90,146,0.93,151,0.85
5,Apple Products,0,0.00,2,1.00,0,0.00,1,1.00,0,...,0,0.00,0,0.00,0,0.00,0,0.00,0,0.00
6,Appliances,37,0.89,26,0.81,37,0.89,25,0.76,25,...,27,0.96,32,0.94,29,0.93,25,0.88,45,0.87
7,Appstore for Android,81,0.72,55,0.76,57,0.81,55,0.80,57,...,62,0.73,69,0.80,67,0.63,38,0.76,60,0.87
8,"Arts, Crafts & Sewing",10,0.90,16,0.94,10,1.00,12,0.92,13,...,18,0.94,18,0.83,9,0.78,15,0.93,9,0.78
9,Automotive,13,0.77,12,0.92,8,0.75,6,1.00,14,...,10,1.00,10,1.00,10,1.00,11,1.00,9,0.78
